In [5]:
import cv2 
import tensorflow as tf
from keras.models import load_model
import pickle
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import numpy as np
from datetime import datetime

new_model = load_model('attendance.h5')

label_dict = {0:'Bill Gates',1:'Jack Ma',2:'Narendra Modi',3:'Elon Musk',4:'Donald Trump'}
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%d/%m/%y %H:%M:%S')
            f.writelines(f'\n{name},{dtString}')
def load_image(filename,new_model):
    img = image.load_img(filename,target_size = (100,100),color_mode = "grayscale")
    img = np.array(img)/255
    img = np.expand_dims(img,axis = 0) 
    img = img.reshape(1,100,100,1)
    result = new_model.predict(img)
    result = list(result[0])
    img_index = result.index(max(result))
    return label_dict[img_index]

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')

vid = cv2.VideoCapture(0) 
  
while(True): 
      
    ret, frame = vid.read() 
    cv2.imwrite("a.jpg", frame)
    pred=load_image('a.jpg',new_model)
    markAttendance(pred)
    font = cv2.FONT_HERSHEY_SIMPLEX 
    
    org = (50, 50) 

    fontScale = 1.5

    color = (255, 255, 0) 

    thickness = 5




    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
        color = (255, 0, 0)  
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
    
    cv2.putText(frame, pred, org, font,  
                       fontScale, (0,0,255), thickness, cv2.LINE_AA) 
    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
vid.release() 
cv2.destroyAllWindows()